## Exploring dataset from https://www.youtube.com/watch?v=tt1xGQ7E7O / https://smashsight.com/download-data/

See what we can use, and first actual goal is to augment the dataset with bounding boxes of the actual game (without the overlay on either side) and train a model to detect this area with the stage they are playing at.

In [2]:
import pandas as pd

# load in datasets
GAME_NAME = 'genesis 5_armada VS leffen'
FILE_NAMES = ['genesis 5_armada VS leffen_1', 'genesis 5_armada VS leffen_2', 'genesis 5_armada VS leffen_3', 'genesis 5_armada VS leffen_4']

df_meta = []
df_frames = []
for i, name in enumerate(FILE_NAMES):
    # First two lines are metadata
    df_meta.append(pd.read_csv('{}/raw_data/{}.txt'.format(GAME_NAME,name), sep=','))
    df_meta[i] = df_meta[i][:1]
    
    df_frames.append(pd.read_csv('{}/raw_data/{}.txt'.format(GAME_NAME,name), sep=',', skiprows=2))

df_meta[0]

,Tournament,LeftPlayerName,RightPlayerName,Player1Char,Player2Char,Player3Char,Player4Char,Level,Winner,GameNumber
0,genesis 5,armada,leffen,Fox_Red,NaN,Fox_Original,NaN,Dreamland,P1:Fox_Red,1


In [23]:
df_frames[0]

,Frame,P1Dmg,P2Dmg,P3Dmg,P4Dmg,P1Stocks,P2Stocks,P3Stocks,P4Stocks,Time
0,0,0,NaN,0,NaN,4,NaN,4,NaN,NaN
1,1,0,NaN,0,NaN,4,NaN,4,NaN,NaN
2,2,0,NaN,0,NaN,4,NaN,4,NaN,NaN
3,3,0,NaN,0,NaN,4,NaN,4,NaN,NaN
4,4,0,NaN,0,NaN,4,NaN,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4974,4974,0,NaN,94,NaN,1,NaN,1,NaN,NaN
4975,4975,0,NaN,94,NaN,1,NaN,1,NaN,NaN
4976,4976,0,NaN,94,NaN,1,NaN,1,NaN,NaN
4977,4977,0,NaN,94,NaN,1,NaN,1,NaN,NaN


The first 11ish seconds of the video are missing from the dataset. Additionally, there are multiple breaks in the video between games that the dataset does not take into account. Each file simply starts at 0 frames and ends at the last frame of the game.

We could either cut the video to match the dataset, or insert frames into the dataset. It would be nice to have the breaks in there because these will be in future videos we make inferences on, and the more data the better. We'll need to try to be as precise as possible in inserting these frames by using the video frames. While the video is 30 fps, we will be using 1 fps for our purposes which will also make this part easier.

In [14]:
dataset_frame_count = sum(df.shape[0] for df in df_frames)
dataset_frame_count

18092

In [15]:
#######################
# get number of frames
import cv2

vid = cv2.VideoCapture('videos/{}.mp4'.format(GAME_NAME))
frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
frame_count - dataset_frame_count # number of frames missing

2899

In [5]:
# todo: extract this to a py script
fps = 30

success, image = vid.read()
count = 1
frame_num = 0

cv2.imwrite('video_frames/{}/frame_%d.jpg'.format(GAME_NAME) % frame_num, image)
frame_num += 1
while success:
    if count % fps == 0:
        cv2.imwrite('video_frames/{}/frame_%d.jpg'.format(GAME_NAME) % frame_num, image)
        frame_num += 1
        if count % 1000 == 0:
            print('Saved frame {}'.format(count))
    success, image = vid.read()
    count += 1
    

Saved frame 3000
Saved frame 6000
Saved frame 9000
Saved frame 12000
Saved frame 15000
Saved frame 18000


Now, cut down the dataset fps as well to match.

In [16]:
df_frames = pd.concat(df_frames).reset_index(drop=True)
del df_frames['Frame']
df_frames

,P1Dmg,P2Dmg,P3Dmg,P4Dmg,P1Stocks,P2Stocks,P3Stocks,P4Stocks,Time
0,0,NaN,0,NaN,4,NaN,4,NaN,NaN
1,0,NaN,0,NaN,4,NaN,4,NaN,NaN
2,0,NaN,0,NaN,4,NaN,4,NaN,NaN
3,0,NaN,0,NaN,4,NaN,4,NaN,NaN
4,0,NaN,0,NaN,4,NaN,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...
18087,105,NaN,13,NaN,1,NaN,3,NaN,NaN
18088,105,NaN,13,NaN,1,NaN,3,NaN,NaN
18089,105,NaN,13,NaN,1,NaN,3,NaN,NaN
18090,105,NaN,13,NaN,1,NaN,3,NaN,NaN


In [17]:
df_frames = df_frames[::fps].reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_frames)

     P1Dmg  P2Dmg  P3Dmg  P4Dmg  P1Stocks  P2Stocks  P3Stocks  P4Stocks  Time
0        0    NaN      0    NaN         4       NaN         4       NaN   NaN
1        0    NaN      0    NaN         4       NaN         4       NaN   NaN
2        0    NaN      0    NaN         4       NaN         4       NaN   NaN
3        0    NaN     16    NaN         4       NaN         4       NaN   NaN
4        0    NaN     18    NaN         4       NaN         4       NaN   NaN
5        0    NaN     24    NaN         4       NaN         4       NaN   NaN
6        0    NaN     29    NaN         4       NaN         4       NaN   NaN
7        0    NaN     54    NaN         4       NaN         4       NaN   NaN
8        0    NaN     54    NaN         4       NaN         4       NaN   NaN
9        0    NaN     55    NaN         4       NaN         4       NaN   NaN
10       0    NaN     68    NaN         4       NaN         4       NaN   NaN
11       0    NaN     83    NaN         4       NaN         4   

In [18]:
# There are 12 missing frames in the front
df_nans = pd.DataFrame(columns=list(df_frames.columns.values), index=range(12))
df_frames = pd.concat([df_nans, df_frames]).reset_index(drop=True)

# Frames after game 1
df_nans = pd.DataFrame(columns=list(df_frames.columns.values), index=range(17))
df_frames = pd.concat([df_frames.iloc[:183], df_nans, df_frames.iloc[183:]]).reset_index(drop=True)

In [19]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_frames)

    P1Dmg P2Dmg P3Dmg P4Dmg P1Stocks P2Stocks P3Stocks P4Stocks Time
0     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
1     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
2     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
3     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
4     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
5     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
6     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
7     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
8     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
9     NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
10    NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
11    NaN   NaN   NaN   NaN      NaN      NaN      NaN      NaN  NaN
12      0   NaN     0   NaN        4      NaN        4      NaN  NaN
13      0   NaN     0   NaN       

## Creating df for stage detection model

At this point, frame by frame, we can see a few instances where the data (e.g. percents) in the dataset did not come as an *exact* match to the video (as we diced it up). In real time you wouldn't be able to tell, but of course it may affect the output of some models. However, right now we are currently only focused on adding the bounding boxes so will have to come back and look at these concerns later if needed, but it doesn't bode well because if it's not synced to the actual video, it would be kinda useless :/ the idea was to use all the other data to extract additional features at a later point. Maybe I'm missing something :thinking: but worst case it could be a good starting point for data gathering if we are willing to go in and resolve the discrepancies. So for now, just add columns for bounding boxes and disregard the other data.

We'll put NaNs for the frames that don't show gameplay, like those with player closeups.

In [28]:
df_frames_stage = pd.DataFrame(columns=['Path', 'Xmin', 'Ymin', 'Xmax', 'Ymax', 'Stage'], index=range(700))
df_frames_stage['Path'] = ['images/frame_' +str(i)+'.jpg' for i, _ in df_frames_stage.iterrows()]
df_frames_stage.loc[12:182, 'Stage'] = 'Dreamland' # game 1
df_frames_stage.loc[200:365, 'Stage'] = 'Final Destination' # game 2
df_frames_stage.loc[383:558, 'Stage'] = 'Battlefield' # game 3
df_frames_stage.loc[564:570, 'Stage'] = 'Battlefield' # some highlights from game 3 - we want the stage to be labelled
df_frames_stage.loc[585:689, 'Stage'] = 'Battlefield' # game 4

# where stage is not null (meaning we see this is gameplay frame), add bounding boxes
bndbox = (102, 0, 536, 360)
df_frames_stage.loc[df_frames_stage['Stage'].notnull(), ['Xmin', 'Ymin', 'Xmax', 'Ymax']] = bndbox

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_frames_stage)

                     Path Xmin Ymin Xmax Ymax              Stage
0      images/frame_0.jpg  NaN  NaN  NaN  NaN                NaN
1      images/frame_1.jpg  NaN  NaN  NaN  NaN                NaN
2      images/frame_2.jpg  NaN  NaN  NaN  NaN                NaN
3      images/frame_3.jpg  NaN  NaN  NaN  NaN                NaN
4      images/frame_4.jpg  NaN  NaN  NaN  NaN                NaN
5      images/frame_5.jpg  NaN  NaN  NaN  NaN                NaN
6      images/frame_6.jpg  NaN  NaN  NaN  NaN                NaN
7      images/frame_7.jpg  NaN  NaN  NaN  NaN                NaN
8      images/frame_8.jpg  NaN  NaN  NaN  NaN                NaN
9      images/frame_9.jpg  NaN  NaN  NaN  NaN                NaN
10    images/frame_10.jpg  NaN  NaN  NaN  NaN                NaN
11    images/frame_11.jpg  NaN  NaN  NaN  NaN                NaN
12    images/frame_12.jpg  102    0  536  360          Dreamland
13    images/frame_13.jpg  102    0  536  360          Dreamland
14    images/frame_14.jpg

In [17]:
# save the annonations
import numpy as np
TRAIN_TEST_SPLIT = .75
mask = np.random.rand(len(df_frames_stage)) < TRAIN_TEST_SPLIT
df_frames_stage[mask].to_csv('{}/train.csv'.format(GAME_NAME), index=False, header=False)
df_frames_stage[~mask].to_csv('{}/test.csv'.format(GAME_NAME), index=False, header=False)

In [18]:
# save the classes
df_classes = pd.DataFrame(df_frames_stage['Stage'].dropna().unique())
df_classes['id'] = df_classes.index
df_classes.to_csv('{}/classes.csv'.format(GAME_NAME), index=False, header=False)

## Train the model

In [20]:
# Download coco pretrained resnet50
import urllib

MODEL_URL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
MODEL_FILE = '../snapshots/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(MODEL_URL, MODEL_FILE)

('../snapshots/resnet50_coco_best_v2.1.0.h5',
 <http.client.HTTPMessage at 0x7f0a4d758c88>)

In [6]:
!python3 ../keras_retinanet/bin/train.py --weights ../snapshots/resnet50_coco_best_v2.1.0.h5 \
--batch-size 8 --steps 100 --epochs 10 \
--snapshot-path ../temp_models --tensorboard-dir tensorboard \
csv genesis\ 5_armada\ VS\ leffen/train.csv genesis\ 5_armada\ VS\ leffen/classes.csv

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

Creating model, this may take a second...






/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 27) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((27,) vs (720,)).
  weight_values[i].shape))
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Model: "retinanet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
_________________________________________________________________________

/home/jupyter/.local/lib/python3.5/site-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/10
2019-12-15 23:16:26.736216: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
100/100 [==============================] - 90s 895ms/step - loss: 1.1942 - regression_loss: 0.2086 - classification_loss: 0.9856

Epoch 00001: saving model to ../models/resnet50_csv_01.h5
Epoch 2/10
100/100 [==============================] - 79s 786ms/step - loss: 0.2274 - regression_loss: 0.0246 - classification_loss: 0.2028

Epoch 00002: saving model to ../models/resnet50_csv_02.h5
Epoch 3/10
100/100 [==============================] - 78s 784ms/step - loss: 0.0675 - regression_loss: 0.0154 - classification_loss: 0.0521

Epoch 00003: saving model to ../models/resnet50_csv_03.h5
Epoch 4/10
100/100 [=================

## Predict and Evaluate

In [9]:
# convert to inference final form
!python3 ../keras_retinanet/bin/convert_model.py ../models_temp/resnet50_csv_10.h5 ../models_final/resnet50_gen5_1.h5

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [32]:
#!python3 ../keras_retinanet/bin/evaluate.py ../models_final/resnet50_gen5_1.h5 \
#    csv genesis\ 5_armada\ VS\ leffen/train.csv genesis\ 5_armada\ VS\ leffen/classes.csv

In [57]:
# todo: make it easier to just look at test set - either split the image directories up there ^ or make predict.py read a csv like train
!python3 ../../predict.py --model ../models_final/resnet50_gen5_1.h5 \
    --labels genesis\ 5_armada\ VS\ leffen/classes.csv \
    --images genesis\ 5_armada\ VS\ leffen/images \
    --output genesis\ 5_armada\ VS\ leffen/output_csv/output_1.csv \
    --confidence 0.5




Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
[INFO] predicting on image 1 of 700
2019-12-17 04:20:04.575938: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2019-12-17 04:20:04.650964: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
[INFO] predicting on image 2 of 700
[INFO] predicting on image 3 of 700
[INFO] predicting on image 

[INFO] predicting on image 197 of 700
[INFO] predicting on image 198 of 700
[INFO] predicting on image 199 of 700
[INFO] predicting on image 200 of 700
[INFO] predicting on image 201 of 700
[INFO] predicting on image 202 of 700
[INFO] predicting on image 203 of 700
[INFO] predicting on image 204 of 700
[INFO] predicting on image 205 of 700
[INFO] predicting on image 206 of 700
[INFO] predicting on image 207 of 700
[INFO] predicting on image 208 of 700
[INFO] predicting on image 209 of 700
[INFO] predicting on image 210 of 700
[INFO] predicting on image 211 of 700
[INFO] predicting on image 212 of 700
[INFO] predicting on image 213 of 700
[INFO] predicting on image 214 of 700
[INFO] predicting on image 215 of 700
[INFO] predicting on image 216 of 700
[INFO] predicting on image 217 of 700
[INFO] predicting on image 218 of 700
[INFO] predicting on image 219 of 700
[INFO] predicting on image 220 of 700
[INFO] predicting on image 221 of 700
[INFO] predicting on image 222 of 700
[INFO] predi

[INFO] predicting on image 413 of 700
[INFO] predicting on image 414 of 700
[INFO] predicting on image 415 of 700
[INFO] predicting on image 416 of 700
[INFO] predicting on image 417 of 700
[INFO] predicting on image 418 of 700
[INFO] predicting on image 419 of 700
[INFO] predicting on image 420 of 700
[INFO] predicting on image 421 of 700
[INFO] predicting on image 422 of 700
[INFO] predicting on image 423 of 700
[INFO] predicting on image 424 of 700
[INFO] predicting on image 425 of 700
[INFO] predicting on image 426 of 700
[INFO] predicting on image 427 of 700
[INFO] predicting on image 428 of 700
[INFO] predicting on image 429 of 700
[INFO] predicting on image 430 of 700
[INFO] predicting on image 431 of 700
[INFO] predicting on image 432 of 700
[INFO] predicting on image 433 of 700
[INFO] predicting on image 434 of 700
[INFO] predicting on image 435 of 700
[INFO] predicting on image 436 of 700
[INFO] predicting on image 437 of 700
[INFO] predicting on image 438 of 700
[INFO] predi

[INFO] predicting on image 629 of 700
[INFO] predicting on image 630 of 700
[INFO] predicting on image 631 of 700
[INFO] predicting on image 632 of 700
[INFO] predicting on image 633 of 700
[INFO] predicting on image 634 of 700
[INFO] predicting on image 635 of 700
[INFO] predicting on image 636 of 700
[INFO] predicting on image 637 of 700
[INFO] predicting on image 638 of 700
[INFO] predicting on image 639 of 700
[INFO] predicting on image 640 of 700
[INFO] predicting on image 641 of 700
[INFO] predicting on image 642 of 700
[INFO] predicting on image 643 of 700
[INFO] predicting on image 644 of 700
[INFO] predicting on image 645 of 700
[INFO] predicting on image 646 of 700
[INFO] predicting on image 647 of 700
[INFO] predicting on image 648 of 700
[INFO] predicting on image 649 of 700
[INFO] predicting on image 650 of 700
[INFO] predicting on image 651 of 700
[INFO] predicting on image 652 of 700
[INFO] predicting on image 653 of 700
[INFO] predicting on image 654 of 700
[INFO] predi

In [ ]:
# visualize
!python3 ../../visualize_batch.py --images genesis\ 5_armada\ VS\ leffen/images \
    --boxes genesis\ 5_armada\ VS\ leffen/output_csv/output_1.csv \
    --output genesis\ 5_armada\ VS\ leffen/output_vid/output_1.mp4

## Predict on a whole new video

In [19]:
# predict str8 on tbh8 video
!python3 ../../predict_video.py --model ../models_final/resnet50_gen5_1.h5 \
    --labels genesis\ 5_armada\ VS\ leffen/classes.csv \
    --video tbh8_leffen_mango/videos/tbh8_leffen_mango.mp4 \
    --output tbh8_leffen_mango/output_vid/output_1.mp4 \
    --confidence 0.5

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
[INFO] predicting on frame 0
2019-12-18 03:48:54.464876: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2019-12-18 03:48:54.536338: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
[INFO] predicting on frame 10
[INFO] predicting on frame 20
[INFO] predicting on frame 30
[INFO] predictin

[INFO] predicting on frame 2330
[INFO] predicting on frame 2340
[INFO] predicting on frame 2350
[INFO] predicting on frame 2360
[INFO] predicting on frame 2370
[INFO] predicting on frame 2380
[INFO] predicting on frame 2390
[INFO] predicting on frame 2400
[INFO] predicting on frame 2410
[INFO] predicting on frame 2420
[INFO] predicting on frame 2430
[INFO] predicting on frame 2440
[INFO] predicting on frame 2450
[INFO] predicting on frame 2460
[INFO] predicting on frame 2470
[INFO] predicting on frame 2480
[INFO] predicting on frame 2490
[INFO] predicting on frame 2500
[INFO] predicting on frame 2510
[INFO] predicting on frame 2520
[INFO] predicting on frame 2530
[INFO] predicting on frame 2540
[INFO] predicting on frame 2550
[INFO] predicting on frame 2560
[INFO] predicting on frame 2570
[INFO] predicting on frame 2580
[INFO] predicting on frame 2590
[INFO] predicting on frame 2600
[INFO] predicting on frame 2610
[INFO] predicting on frame 2620
[INFO] predicting on frame 2630
[INFO] p

[INFO] predicting on frame 4900
[INFO] predicting on frame 4910
[INFO] predicting on frame 4920
[INFO] predicting on frame 4930
[INFO] predicting on frame 4940
[INFO] predicting on frame 4950
[INFO] predicting on frame 4960
[INFO] predicting on frame 4970
[INFO] predicting on frame 4980
[INFO] predicting on frame 4990
[INFO] predicting on frame 5000
[INFO] predicting on frame 5010
[INFO] predicting on frame 5020
[INFO] predicting on frame 5030
[INFO] predicting on frame 5040
[INFO] predicting on frame 5050
[INFO] predicting on frame 5060
[INFO] predicting on frame 5070
[INFO] predicting on frame 5080
[INFO] predicting on frame 5090
[INFO] predicting on frame 5100
[INFO] predicting on frame 5110
[INFO] predicting on frame 5120
[INFO] predicting on frame 5130
[INFO] predicting on frame 5140
[INFO] predicting on frame 5150
[INFO] predicting on frame 5160
[INFO] predicting on frame 5170
[INFO] predicting on frame 5180
[INFO] predicting on frame 5190
[INFO] predicting on frame 5200
[INFO] p

[INFO] predicting on frame 7470
[INFO] predicting on frame 7480
[INFO] predicting on frame 7490
[INFO] predicting on frame 7500
[INFO] predicting on frame 7510
[INFO] predicting on frame 7520
[INFO] predicting on frame 7530
[INFO] predicting on frame 7540
[INFO] predicting on frame 7550
[INFO] predicting on frame 7560
[INFO] predicting on frame 7570
[INFO] predicting on frame 7580
[INFO] predicting on frame 7590
[INFO] predicting on frame 7600
[INFO] predicting on frame 7610
[INFO] predicting on frame 7620
[INFO] predicting on frame 7630
[INFO] predicting on frame 7640
[INFO] predicting on frame 7650
[INFO] predicting on frame 7660
[INFO] predicting on frame 7670
[INFO] predicting on frame 7680
[INFO] predicting on frame 7690
[INFO] predicting on frame 7700
[INFO] predicting on frame 7710
[INFO] predicting on frame 7720
[INFO] predicting on frame 7730
[INFO] predicting on frame 7740
[INFO] predicting on frame 7750
[INFO] predicting on frame 7760
[INFO] predicting on frame 7770
[INFO] p

...

[INFO] predicting on frame 12520
[INFO] predicting on frame 12530
[INFO] predicting on frame 12540
[INFO] predicting on frame 12550
[INFO] predicting on frame 12560
[INFO] predicting on frame 12570
[INFO] predicting on frame 12580
[INFO] predicting on frame 12590
[INFO] predicting on frame 12600
[INFO] predicting on frame 12610
[INFO] predicting on frame 12620
[INFO] predicting on frame 12630
[INFO] predicting on frame 12640
[INFO] predicting on frame 12650
[INFO] predicting on frame 12660
[INFO] predicting on frame 12670
[INFO] predicting on frame 12680
[INFO] predicting on frame 12690
[INFO] predicting on frame 12700
[INFO] predicting on frame 12710
[INFO] predicting on frame 12720
[INFO] predicting on frame 12730
[INFO] predicting on frame 12740
[INFO] predicting on frame 12750
[INFO] predicting on frame 12760
[INFO] predicting on frame 12770
[INFO] predicting on frame 12780
[INFO] predicting on frame 12790
[INFO] predicting on frame 12800
[INFO] predicting on frame 12810
[INFO] pre

[INFO] predicting on frame 15010
[INFO] predicting on frame 15020
[INFO] predicting on frame 15030
[INFO] predicting on frame 15040
[INFO] predicting on frame 15050
[INFO] predicting on frame 15060
[INFO] predicting on frame 15070
[INFO] predicting on frame 15080
[INFO] predicting on frame 15090
[INFO] predicting on frame 15100
[INFO] predicting on frame 15110
[INFO] predicting on frame 15120
[INFO] predicting on frame 15130
[INFO] predicting on frame 15140
[INFO] predicting on frame 15150
[INFO] predicting on frame 15160
[INFO] predicting on frame 15170
[INFO] predicting on frame 15180
[INFO] predicting on frame 15190
[INFO] predicting on frame 15200
[INFO] predicting on frame 15210
[INFO] predicting on frame 15220
[INFO] predicting on frame 15230
[INFO] predicting on frame 15240
[INFO] predicting on frame 15250
[INFO] predicting on frame 15260
[INFO] predicting on frame 15270
[INFO] predicting on frame 15280
[INFO] predicting on frame 15290
[INFO] predicting on frame 15300
[INFO] pre

[INFO] predicting on frame 17500
[INFO] predicting on frame 17510
[INFO] predicting on frame 17520
[INFO] predicting on frame 17530
[INFO] predicting on frame 17540
[INFO] predicting on frame 17550
[INFO] predicting on frame 17560
[INFO] predicting on frame 17570
[INFO] predicting on frame 17580
[INFO] predicting on frame 17590
[INFO] predicting on frame 17600
[INFO] predicting on frame 17610
[INFO] predicting on frame 17620
[INFO] predicting on frame 17630
[INFO] predicting on frame 17640
[INFO] predicting on frame 17650
[INFO] predicting on frame 17660
[INFO] predicting on frame 17670
[INFO] predicting on frame 17680
[INFO] predicting on frame 17690
[INFO] predicting on frame 17700
[INFO] predicting on frame 17710
[INFO] predicting on frame 17720
[INFO] predicting on frame 17730
[INFO] predicting on frame 17740
[INFO] predicting on frame 17750
[INFO] predicting on frame 17760
[INFO] predicting on frame 17770
[INFO] predicting on frame 17780
[INFO] predicting on frame 17790
[INFO] pre

[INFO] predicting on frame 19990
[INFO] predicting on frame 20000
[INFO] predicting on frame 20010
[INFO] predicting on frame 20020
[INFO] predicting on frame 20030
[INFO] predicting on frame 20040
[INFO] predicting on frame 20050
[INFO] predicting on frame 20060
[INFO] predicting on frame 20070
[INFO] predicting on frame 20080
[INFO] predicting on frame 20090
[INFO] predicting on frame 20100
[INFO] predicting on frame 20110
[INFO] predicting on frame 20120
[INFO] predicting on frame 20130
[INFO] predicting on frame 20140
[INFO] predicting on frame 20150
[INFO] predicting on frame 20160
[INFO] predicting on frame 20170
[INFO] predicting on frame 20180
[INFO] predicting on frame 20190
[INFO] predicting on frame 20200
[INFO] predicting on frame 20210
[INFO] predicting on frame 20220
[INFO] predicting on frame 20230
[INFO] predicting on frame 20240
[INFO] predicting on frame 20250
[INFO] predicting on frame 20260
[INFO] predicting on frame 20270
[INFO] predicting on frame 20280
[INFO] pre

[INFO] predicting on frame 22480
[INFO] predicting on frame 22490
[INFO] predicting on frame 22500
[INFO] predicting on frame 22510
[INFO] predicting on frame 22520
[INFO] predicting on frame 22530
[INFO] predicting on frame 22540


## Try same flow including addition of new objects: player char

In [ ]:
df_frames_stage = pd.DataFrame(columns=['Path', 'Xmin', 'Ymin', 'Xmax', 'Ymax', 'Stage'], index=range(700))
df_frames_stage['Path'] = ['images/frame_' +str(i)+'.jpg' for i, _ in df_frames_stage.iterrows()]
df_frames_stage.loc[12:182, 'Stage'] = 'Dreamland' # game 1
df_frames_stage.loc[200:365, 'Stage'] = 'Final Destination' # game 2
df_frames_stage.loc[383:558, 'Stage'] = 'Battlefield' # game 3
df_frames_stage.loc[564:570, 'Stage'] = 'Battlefield' # some highlights from game 3 - we want the stage to be labelled
df_frames_stage.loc[585:689, 'Stage'] = 'Battlefield' # game 4

# where stage is not null (meaning we see this is gameplay frame), add bounding boxes
bndbox = (102, 0, 536, 360)
df_frames_stage.loc[df_frames_stage['Stage'].notnull(), ['Xmin', 'Ymin', 'Xmax', 'Ymax']] = bndbox

In [29]:
# P1
df_frames_P1Char = pd.DataFrame(columns=['Path','P1CharXmin', 'P1CharYmin','P1CharXmax','P1CharYmax', 'P1Char'], index=range(700))
df_frames_P1Char['Path'] = ['images/frame_' +str(i)+'.jpg' for i, _ in df_frames_P1Char.iterrows()]
# add player char info
df_frames_P1Char.loc[12:182, 'P1Char'] = 'Fox_Red' # game 1
df_frames_P1Char.loc[200:365, 'P1Char'] = 'Fox_Red' # game 2
df_frames_P1Char.loc[383:558, 'P1Char'] = 'Fox_Red' # game 3
df_frames_P1Char.loc[564:570, 'P1Char'] = 'Fox_Red' # some highlights from game 3 - we want the stage to be labelled
df_frames_P1Char.loc[585:689, 'P1Char'] = 'Fox_Red' # game 4

# where stage is not null (meaning we see this is a gameplay frame), add bounding boxes
p1CharBndbox = (115, 274, 133, 295)
df_frames_P1Char.loc[df_frames_stage['Stage'].notnull(), ['P1CharXmin', 'P1CharYmin','P1CharXmax','P1CharYmax']] = p1CharBndbox

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_frames_P1Char)

                     Path P1CharXmin P1CharYmin P1CharXmax P1CharYmax   P1Char
0      images/frame_0.jpg        NaN        NaN        NaN        NaN      NaN
1      images/frame_1.jpg        NaN        NaN        NaN        NaN      NaN
2      images/frame_2.jpg        NaN        NaN        NaN        NaN      NaN
3      images/frame_3.jpg        NaN        NaN        NaN        NaN      NaN
4      images/frame_4.jpg        NaN        NaN        NaN        NaN      NaN
5      images/frame_5.jpg        NaN        NaN        NaN        NaN      NaN
6      images/frame_6.jpg        NaN        NaN        NaN        NaN      NaN
7      images/frame_7.jpg        NaN        NaN        NaN        NaN      NaN
8      images/frame_8.jpg        NaN        NaN        NaN        NaN      NaN
9      images/frame_9.jpg        NaN        NaN        NaN        NaN      NaN
10    images/frame_10.jpg        NaN        NaN        NaN        NaN      NaN
11    images/frame_11.jpg        NaN        NaN     

In [30]:
# P3
df_frames_P3Char = pd.DataFrame(columns=['Path','P3CharXmin', 'P3CharYmin','P3CharXmax','P3CharYmax', 'P3Char'], index=range(700))
df_frames_P3Char['Path'] = ['images/frame_' +str(i)+'.jpg' for i, _ in df_frames_P3Char.iterrows()]
# add player char info
df_frames_P3Char.loc[12:182, 'P3Char'] = 'Fox_Original' # game 1
df_frames_P3Char.loc[200:365, 'P3Char'] = 'Fox_Original' # game 2
df_frames_P3Char.loc[383:558, 'P3Char'] = 'Fox_Original' # game 3
df_frames_P3Char.loc[564:570, 'P3Char'] = 'Fox_Original' # some highlights from game 3 - we want the stage to be labelled
df_frames_P3Char.loc[585:689, 'P3Char'] = 'Fox_Original' # game 4

# where stage is not null (meaning we see this is a gameplay frame), add bounding boxes
p3CharBndbox = (324, 274, 343, 295)
df_frames_P3Char.loc[df_frames_stage['Stage'].notnull(), ['P3CharXmin', 'P3CharYmin','P3CharXmax','P3CharYmax']] = p3CharBndbox

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_frames_P3Char)

                     Path P3CharXmin P3CharYmin P3CharXmax P3CharYmax  \
0      images/frame_0.jpg        NaN        NaN        NaN        NaN   
1      images/frame_1.jpg        NaN        NaN        NaN        NaN   
2      images/frame_2.jpg        NaN        NaN        NaN        NaN   
3      images/frame_3.jpg        NaN        NaN        NaN        NaN   
4      images/frame_4.jpg        NaN        NaN        NaN        NaN   
5      images/frame_5.jpg        NaN        NaN        NaN        NaN   
6      images/frame_6.jpg        NaN        NaN        NaN        NaN   
7      images/frame_7.jpg        NaN        NaN        NaN        NaN   
8      images/frame_8.jpg        NaN        NaN        NaN        NaN   
9      images/frame_9.jpg        NaN        NaN        NaN        NaN   
10    images/frame_10.jpg        NaN        NaN        NaN        NaN   
11    images/frame_11.jpg        NaN        NaN        NaN        NaN   
12    images/frame_12.jpg        324        274    

In [31]:
# combine dfs
final_df = pd.DataFrame(np.concatenate([df_frames_stage.values, df_frames_P1Char.values, df_frames_P3Char.values]), columns=df_frames_stage.columns)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(final_df)

                      Path Xmin Ymin Xmax Ymax              Stage
0       images/frame_0.jpg  NaN  NaN  NaN  NaN                NaN
1       images/frame_1.jpg  NaN  NaN  NaN  NaN                NaN
2       images/frame_2.jpg  NaN  NaN  NaN  NaN                NaN
3       images/frame_3.jpg  NaN  NaN  NaN  NaN                NaN
4       images/frame_4.jpg  NaN  NaN  NaN  NaN                NaN
5       images/frame_5.jpg  NaN  NaN  NaN  NaN                NaN
6       images/frame_6.jpg  NaN  NaN  NaN  NaN                NaN
7       images/frame_7.jpg  NaN  NaN  NaN  NaN                NaN
8       images/frame_8.jpg  NaN  NaN  NaN  NaN                NaN
9       images/frame_9.jpg  NaN  NaN  NaN  NaN                NaN
10     images/frame_10.jpg  NaN  NaN  NaN  NaN                NaN
11     images/frame_11.jpg  NaN  NaN  NaN  NaN                NaN
12     images/frame_12.jpg  102    0  536  360          Dreamland
13     images/frame_13.jpg  102    0  536  360          Dreamland
14     ima

In [32]:
# save the annonations
import numpy as np
TRAIN_TEST_SPLIT = .75
mask = np.random.rand(len(final_df)) < TRAIN_TEST_SPLIT
final_df[mask].to_csv('{}/train_2.csv'.format(GAME_NAME), index=False, header=False)
final_df[~mask].to_csv('{}/test_2.csv'.format(GAME_NAME), index=False, header=False)

In [33]:
# save the classes
df_classes = pd.DataFrame(final_df['Stage'].dropna().unique())
df_classes['id'] = df_classes.index
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_classes)
df_classes.to_csv('{}/classes_2.csv'.format(GAME_NAME), index=False, header=False)

                   0  id
0          Dreamland   0
1  Final Destination   1
2        Battlefield   2
3            Fox_Red   3
4       Fox_Original   4


In [34]:
!python3 ../keras_retinanet/bin/train.py --weights ../snapshots/resnet50_coco_best_v2.1.0.h5 \
--batch-size 4 --steps 525 --epochs 10 \
--snapshot-path ../models_temp/ --tensorboard-dir tensorboard \
csv genesis\ 5_armada\ VS\ leffen/train_2.csv genesis\ 5_armada\ VS\ leffen/classes_2.csv

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

Creating model, this may take a second...






/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((3, 3, 256, 45) vs (720, 256, 3, 3)).
  weight_values[i].shape))
/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:1281: UserWarning: Skipping loading of weights for layer classification_submodel due to mismatch in shape ((45,) vs (720,)).
  weight_values[i].shape))
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Model: "retinanet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
_________________________________________________________________________

/home/jupyter/.local/lib/python3.5/site-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/10
2019-12-19 01:50:07.900816: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
525/525 [==============================] - 412s 784ms/step - loss: 0.6663 - regression_loss: 0.1807 - classification_loss: 0.4856

Epoch 00001: saving model to ../models_temp/resnet50_csv_01.h5
Epoch 2/10
525/525 [==============================] - 404s 770ms/step - loss: 0.1389 - regression_loss: 0.0284 - classification_loss: 0.1105

Epoch 00002: saving model to ../models_temp/resnet50_csv_02.h5
Epoch 3/10
525/525 [==============================] - 404s 770ms/step - loss: 0.0931 - regression_loss: 0.0148 - classification_loss: 0.0783

Epoch 00003: saving model to ../models_temp/resnet50_csv_03.h5
Epoch 4/10
525/525 

In [36]:
!python3 ../keras_retinanet/bin/convert_model.py ../models_temp/resnet50_csv_07.h5 ../models_final/resnet50_gen5_2.h5

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [37]:
!python3 ../../predict_batch.py --model ../models_final/resnet50_gen5_2.h5 \
    --labels genesis\ 5_armada\ VS\ leffen/classes_2.csv \
    --images genesis\ 5_armada\ VS\ leffen/images \
    --output genesis\ 5_armada\ VS\ leffen/output_csv/output_2.csv \
    --confidence 0.5

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
[INFO] predicting on image 1 of 700
2019-12-19 02:46:43.353175: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2019-12-19 02:46:43.433583: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
[INFO] predicting on image 2 of 700
[INFO] predicting on image 3 of 700
[INFO] predicting on image 

[INFO] predicting on image 197 of 700
[INFO] predicting on image 198 of 700
[INFO] predicting on image 199 of 700
[INFO] predicting on image 200 of 700
[INFO] predicting on image 201 of 700
[INFO] predicting on image 202 of 700
[INFO] predicting on image 203 of 700
[INFO] predicting on image 204 of 700
[INFO] predicting on image 205 of 700
[INFO] predicting on image 206 of 700
[INFO] predicting on image 207 of 700
[INFO] predicting on image 208 of 700
[INFO] predicting on image 209 of 700
[INFO] predicting on image 210 of 700
[INFO] predicting on image 211 of 700
[INFO] predicting on image 212 of 700
[INFO] predicting on image 213 of 700
[INFO] predicting on image 214 of 700
[INFO] predicting on image 215 of 700
[INFO] predicting on image 216 of 700
[INFO] predicting on image 217 of 700
[INFO] predicting on image 218 of 700
[INFO] predicting on image 219 of 700
[INFO] predicting on image 220 of 700
[INFO] predicting on image 221 of 700
[INFO] predicting on image 222 of 700
[INFO] predi

[INFO] predicting on image 413 of 700
[INFO] predicting on image 414 of 700
[INFO] predicting on image 415 of 700
[INFO] predicting on image 416 of 700
[INFO] predicting on image 417 of 700
[INFO] predicting on image 418 of 700
[INFO] predicting on image 419 of 700
[INFO] predicting on image 420 of 700
[INFO] predicting on image 421 of 700
[INFO] predicting on image 422 of 700
[INFO] predicting on image 423 of 700
[INFO] predicting on image 424 of 700
[INFO] predicting on image 425 of 700
[INFO] predicting on image 426 of 700
[INFO] predicting on image 427 of 700
[INFO] predicting on image 428 of 700
[INFO] predicting on image 429 of 700
[INFO] predicting on image 430 of 700
[INFO] predicting on image 431 of 700
[INFO] predicting on image 432 of 700
[INFO] predicting on image 433 of 700
[INFO] predicting on image 434 of 700
[INFO] predicting on image 435 of 700
[INFO] predicting on image 436 of 700
[INFO] predicting on image 437 of 700
[INFO] predicting on image 438 of 700
[INFO] predi

[INFO] predicting on image 629 of 700
[INFO] predicting on image 630 of 700
[INFO] predicting on image 631 of 700
[INFO] predicting on image 632 of 700
[INFO] predicting on image 633 of 700
[INFO] predicting on image 634 of 700
[INFO] predicting on image 635 of 700
[INFO] predicting on image 636 of 700
[INFO] predicting on image 637 of 700
[INFO] predicting on image 638 of 700
[INFO] predicting on image 639 of 700
[INFO] predicting on image 640 of 700
[INFO] predicting on image 641 of 700
[INFO] predicting on image 642 of 700
[INFO] predicting on image 643 of 700
[INFO] predicting on image 644 of 700
[INFO] predicting on image 645 of 700
[INFO] predicting on image 646 of 700
[INFO] predicting on image 647 of 700
[INFO] predicting on image 648 of 700
[INFO] predicting on image 649 of 700
[INFO] predicting on image 650 of 700
[INFO] predicting on image 651 of 700
[INFO] predicting on image 652 of 700
[INFO] predicting on image 653 of 700
[INFO] predicting on image 654 of 700
[INFO] predi

In [49]:
# visualize
!python3 ../../visualize_batch.py --images genesis\ 5_armada\ VS\ leffen/images \
    --boxes genesis\ 5_armada\ VS\ leffen/output_csv/output_2.csv \
    --output genesis\ 5_armada\ VS\ leffen/output_vid/output_2.mp4

genesis 5_armada VS leffen/images/frame_0.jpg
genesis 5_armada VS leffen/images/frame_1.jpg
genesis 5_armada VS leffen/images/frame_2.jpg
genesis 5_armada VS leffen/images/frame_3.jpg
genesis 5_armada VS leffen/images/frame_4.jpg
genesis 5_armada VS leffen/images/frame_5.jpg
genesis 5_armada VS leffen/images/frame_6.jpg
genesis 5_armada VS leffen/images/frame_7.jpg
genesis 5_armada VS leffen/images/frame_8.jpg
genesis 5_armada VS leffen/images/frame_9.jpg
genesis 5_armada VS leffen/images/frame_10.jpg
genesis 5_armada VS leffen/images/frame_11.jpg
genesis 5_armada VS leffen/images/frame_12.jpg
genesis 5_armada VS leffen/images/frame_13.jpg
genesis 5_armada VS leffen/images/frame_14.jpg
genesis 5_armada VS leffen/images/frame_15.jpg
genesis 5_armada VS leffen/images/frame_16.jpg
genesis 5_armada VS leffen/images/frame_17.jpg
genesis 5_armada VS leffen/images/frame_18.jpg
genesis 5_armada VS leffen/images/frame_19.jpg
genesis 5_armada VS leffen/images/frame_20.jpg
genesis 5_armada VS lef

genesis 5_armada VS leffen/images/frame_174.jpg
genesis 5_armada VS leffen/images/frame_175.jpg
genesis 5_armada VS leffen/images/frame_176.jpg
genesis 5_armada VS leffen/images/frame_177.jpg
genesis 5_armada VS leffen/images/frame_178.jpg
genesis 5_armada VS leffen/images/frame_179.jpg
genesis 5_armada VS leffen/images/frame_180.jpg
genesis 5_armada VS leffen/images/frame_181.jpg
genesis 5_armada VS leffen/images/frame_182.jpg
genesis 5_armada VS leffen/images/frame_183.jpg
genesis 5_armada VS leffen/images/frame_184.jpg
genesis 5_armada VS leffen/images/frame_185.jpg
genesis 5_armada VS leffen/images/frame_186.jpg
genesis 5_armada VS leffen/images/frame_187.jpg
genesis 5_armada VS leffen/images/frame_188.jpg
genesis 5_armada VS leffen/images/frame_189.jpg
genesis 5_armada VS leffen/images/frame_190.jpg
genesis 5_armada VS leffen/images/frame_191.jpg
genesis 5_armada VS leffen/images/frame_192.jpg
genesis 5_armada VS leffen/images/frame_193.jpg
genesis 5_armada VS leffen/images/frame_

genesis 5_armada VS leffen/images/frame_346.jpg
genesis 5_armada VS leffen/images/frame_347.jpg
genesis 5_armada VS leffen/images/frame_348.jpg
genesis 5_armada VS leffen/images/frame_349.jpg
genesis 5_armada VS leffen/images/frame_350.jpg
genesis 5_armada VS leffen/images/frame_351.jpg
genesis 5_armada VS leffen/images/frame_352.jpg
genesis 5_armada VS leffen/images/frame_353.jpg
genesis 5_armada VS leffen/images/frame_354.jpg
genesis 5_armada VS leffen/images/frame_355.jpg
genesis 5_armada VS leffen/images/frame_356.jpg
genesis 5_armada VS leffen/images/frame_357.jpg
genesis 5_armada VS leffen/images/frame_358.jpg
genesis 5_armada VS leffen/images/frame_359.jpg
genesis 5_armada VS leffen/images/frame_360.jpg
genesis 5_armada VS leffen/images/frame_361.jpg
genesis 5_armada VS leffen/images/frame_362.jpg
genesis 5_armada VS leffen/images/frame_363.jpg
genesis 5_armada VS leffen/images/frame_364.jpg
genesis 5_armada VS leffen/images/frame_365.jpg
genesis 5_armada VS leffen/images/frame_

genesis 5_armada VS leffen/images/frame_522.jpg
genesis 5_armada VS leffen/images/frame_523.jpg
genesis 5_armada VS leffen/images/frame_524.jpg
genesis 5_armada VS leffen/images/frame_525.jpg
genesis 5_armada VS leffen/images/frame_526.jpg
genesis 5_armada VS leffen/images/frame_527.jpg
genesis 5_armada VS leffen/images/frame_528.jpg
genesis 5_armada VS leffen/images/frame_529.jpg
genesis 5_armada VS leffen/images/frame_530.jpg
genesis 5_armada VS leffen/images/frame_531.jpg
genesis 5_armada VS leffen/images/frame_532.jpg
genesis 5_armada VS leffen/images/frame_533.jpg
genesis 5_armada VS leffen/images/frame_534.jpg
genesis 5_armada VS leffen/images/frame_535.jpg
genesis 5_armada VS leffen/images/frame_536.jpg
genesis 5_armada VS leffen/images/frame_537.jpg
genesis 5_armada VS leffen/images/frame_538.jpg
genesis 5_armada VS leffen/images/frame_539.jpg
genesis 5_armada VS leffen/images/frame_540.jpg
genesis 5_armada VS leffen/images/frame_541.jpg
genesis 5_armada VS leffen/images/frame_

In [70]:
# predict on tbh8
!python3 ../../predict_video.py --model ../models_final/resnet50_gen5_2.h5 \
    --labels genesis\ 5_armada\ VS\ leffen/classes_2.csv \
    --video tbh8_leffen_mango/videos/tbh8_leffen_mango.mp4 \
    --output tbh8_leffen_mango/output_vid/output_tbh8_2.mp4 \
    --confidence 0.09

Using TensorFlow backend.
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/py

/home/jupyter/.local/lib/python3.5/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
[INFO] predicting on frame 0
2019-12-19 04:24:42.582473: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2019-12-19 04:24:42.659932: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudnn.so.7
LABEL Dreamland
[INFO] predicting on frame 10
LABEL Dreamland
[INFO] predicting on frame 20
LABEL Dreamlan